In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import shap

In [ ]:
config_file_path = 'config.yml' 
config = load_config(config_file_path)

df = pd.read_csv("/Users/afedynak/BAARD/master_sheet.csv")
df.head()

# Define target and features
target_col = 'week10_remission_flag'
X = df.drop(columns=[target_col])  # Features
y = df[target_col]  # Target variable

# Identify categorical and numerical columns
categorical_features = X.select_dtypes(include=['object']).columns  # Categorical columns
numerical_features = X.select_dtypes(exclude=['object']).columns  # Numerical columns

# Define the columns (for reference)
columns = ['age', 'madrs', 'phq9', 'gender', 'race', 'ethnicity', 'education_level', 
           'medication_group', 'remission_status', 'AIS_01', 'LIS_01', 'MDMIS_01', 'MVCIS_01', 
           'IMIS_01', 'IL.6', 'gp130', 'IL.8.CXCL.8', 'MIF', 'CCL.2.MCP.1', 'IL.1beta.IL.1F2', 
           'CCL.20.MIP.3alpha', 'CCL.4.MIP.1beta', 'GM.CSF']

categorical = ['gender', 'race', 'ethnicity', 'medication_group', 'remission_status', 'education_level']
numerical = [col for col in columns if col not in categorical]

# Preprocessing pipeline (scaling numeric and encoding categorical features)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [ ]:
# Outer loop cross-validation (StratifiedKFold)
outer_cv = StratifiedKFold(
    config['cross_validation']['n_splits'], 
    shuffle = config['cross_validation']['shuffle'], 
    random_state = config['cross_validation']['random_state'])

In [ ]:
# Hyperparameter grid for inner loop (GridSearchCV)
param_grid = {
    'classifier__C': [0.1, 1, 10],  # Regularization strength
    'classifier__l1_ratio': [0.1, 0.5, 0.9]  # ElasticNet mixing parameter
}

In [ ]:
# Start the nested cross-validation
outer_results = []  # To store the results from the outer loop

for outer_train_idx, outer_test_idx in outer_cv.split(X, y):
    # Split data into outer train and test sets
    X_train_outer, X_test_outer = X.iloc[outer_train_idx], X.iloc[outer_test_idx]
    y_train_outer, y_test_outer = y.iloc[outer_train_idx], y.iloc[outer_test_idx]

    # Inner loop cross-validation for hyperparameter tuning (GridSearchCV)
    inner_cv = StratifiedKFold(
        n_splits = config['cross_validation']['n_splits'], 
        shuffle = config['cross_validation']['shuffle'],
        random_state = config['cross_validation']['random_state'])
    
    # Create a pipeline with preprocessor and logistic regression model
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            penalty = config['model_params']['penalty'],
            solver = config['model_params']['solver'], 
            max_iter = config['model_params']['max_iter'], 
            random_state = config['model_params']['random_state']
        ))
    ])
    
    # GridSearchCV to tune hyperparameters on the inner fold
    grid_search = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)
    
    # Fit GridSearchCV on the outer training data
    grid_search.fit(X_train_outer, y_train_outer)
    
    # Get the best model from the grid search
    best_model = grid_search.best_estimator_
    
    # Evaluate the model on the outer test data
    y_pred_outer = best_model.predict(X_test_outer)
    y_pred_proba_outer = best_model.predict_proba(X_test_outer)[:, 1]
    
    # Calculate metrics for this outer fold
    accuracy = accuracy_score(y_test_outer, y_pred_outer)
    roc_auc = roc_auc_score(y_test_outer, y_pred_proba_outer)
    
    # Store the results for each outer fold
    outer_results.append((accuracy, roc_auc))
    
    # Print the results of this outer fold
    print(f"Outer fold results - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")
    print(f"Best Hyperparameters: {grid_search.best_params_}")

In [ ]:
# After looping through all outer folds, calculate the mean performance
outer_accuracies = [result[0] for result in outer_results]
outer_roc_auc = [result[1] for result in outer_results]

print(f"\nOverall Cross-Validation Accuracy: {np.mean(outer_accuracies):.4f}")
print(f"Overall ROC-AUC: {np.mean(outer_roc_auc):.4f}")
print(f"Standard Deviation of Accuracy: {np.std(outer_accuracies):.4f}")
print(f"Standard Deviation of ROC-AUC: {np.std(outer_roc_auc):.4f}")

In [ ]:
# SHAP Analysis for model interpretability (using the best model from grid search)
X_sample = X.sample(n=100, random_state=42)  # Sample 100 rows for SHAP analysis
explainer = shap.Explainer(best_model, preprocessor.transform(X_sample))
shap_values = explainer(X_sample)

# Plot SHAP summary
shap.summary_plot(shap_values, X_sample)